# Chapter 3 Exercise 2: Uplift Tree - 为因果推断而生的决策树

## 学习目标

1. 理解 Uplift Tree 的分裂准则
2. 实现简单的 Uplift 增益计算
3. 理解与传统决策树的区别
4. 掌握叶节点的 Uplift 估计

---

## 从普通决策树到 Uplift Tree

### 传统决策树的目标

普通决策树想要回答：**对于特征 X 的人，结果 Y 是多少？**

它的分裂目标是让叶节点内的 Y 尽可能 "纯" (同质)。

### Uplift Tree 的目标

Uplift Tree 想要回答一个更高级的问题：**对于特征 X 的人，处理带来的增量效果是多少？**

它的分裂目标是让叶节点内的 **处理效应** 尽可能同质，同时让不同叶节点的效应差异最大！

## 生活化类比：营销活动的精准投放

假设你是电商平台的运营，要发放优惠券：

**传统决策树思路**：找出 "谁最可能购买" → 给购买概率高的人发券

问题是：购买概率高的人可能 **本来就会买**！给他们发券是浪费钱。

**Uplift Tree 思路**：找出 "发券后增量购买最大的人" → 精准投放

| 用户类型 | 不发券购买率 | 发券购买率 | Uplift | 发券效果 |
|---------|------------|----------|--------|--------|
| 铁粉 | 90% | 92% | +2% | 浪费钱! |
| 潜力股 | 20% | 50% | **+30%** | 值得投! |
| 流失用户 | 5% | 6% | +1% | 收益低 |

Uplift Tree 能自动找出 "潜力股" 这类人群！

In [ ]:
# 导入必要的库
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import Tuple, Optional

# 设置随机种子
np.random.seed(42)
print("环境准备完成！")

---

## Part 1: Uplift 增益计算

### 什么是 Uplift？

对于一组样本，Uplift 定义为：

$$Uplift = P(Y=1|T=1) - P(Y=1|T=0)$$

即：**处理组的转化率 - 控制组的转化率**

### Uplift 的取值范围

| Uplift 值 | 含义 |
|-----------|------|
| > 0 | 处理有正向效果 (值得做) |
| = 0 | 处理没有效果 (可做可不做) |
| < 0 | 处理有负向效果 (千万别做!) |

In [ ]:
# 练习 2.1: Uplift 增益计算

def calculate_simple_uplift(
    y: np.ndarray,
    t: np.ndarray
) -> float:
    """
    计算简单的 Uplift (处理组转化率 - 控制组转化率)
    
    TODO: 实现 Uplift 计算
    
    Args:
        y: 结果 (0/1)
        t: 处理状态 (0/1)
    
    Returns:
        uplift value
    """
    # TODO: 分离处理组和控制组
    mask_t = t == 1
    mask_c = t == 0
    
    # TODO: 处理边界情况
    if mask_t.sum() == 0 or mask_c.sum() == 0:
        return 0.0
    
    # TODO: 计算处理组的平均结果 (转化率)
    # 你的代码
    
    # TODO: 计算控制组的平均结果 (转化率)
    # 你的代码
    
    # TODO: 返回差值
    # 你的代码
    pass

# 测试
y_test = np.array([1, 1, 0, 1, 0, 0, 1, 0])
t_test = np.array([1, 1, 1, 1, 0, 0, 0, 0])
uplift = calculate_simple_uplift(y_test, t_test)
if uplift is not None:
    print(f"处理组转化率: {y_test[t_test==1].mean():.2%}")
    print(f"控制组转化率: {y_test[t_test==0].mean():.2%}")
    print(f"Uplift: {uplift:.2%}")
else:
    print("[未完成] 请完成 calculate_simple_uplift 函数")

---

## Part 2: 分裂准则 - KL 散度和欧氏距离

### 为什么需要特殊的分裂准则？

传统决策树用 **信息增益** 或 **基尼系数** 来分裂，目标是预测 Y。

但 Uplift Tree 要预测的是 **处理效应**，需要衡量处理组和控制组的 **分布差异**。

### 常用的分裂准则

#### 1. KL 散度 (Kullback-Leibler Divergence)

衡量两个概率分布的差异：

$$D_{KL} = p_t \log\frac{p_t}{p_c} + (1-p_t) \log\frac{1-p_t}{1-p_c}$$

其中 $p_t$ 是处理组转化率，$p_c$ 是控制组转化率。

**直觉**：如果处理组和控制组的转化率分布差异大，说明处理有效果！

#### 2. 欧氏距离 (Euclidean Distance)

$$ED = (p_t - p_c)^2$$

**直觉**：Uplift 的平方，简单直接。

In [ ]:
# 练习 2.1 续: KL 散度增益

def calculate_kl_divergence_gain(
    y: np.ndarray,
    t: np.ndarray
) -> float:
    """
    计算 KL 散度增益 (Kullback-Leibler Divergence)
    
    KL 散度衡量处理组和控制组的分布差异:
    D_KL = p_t * log(p_t / p_c) + (1 - p_t) * log((1 - p_t) / (1 - p_c))
    
    TODO: 实现 KL 散度计算
    
    Args:
        y: 结果
        t: 处理状态
    
    Returns:
        KL divergence value
    """
    mask_t = t == 1
    mask_c = t == 0
    
    n_t = mask_t.sum()
    n_c = mask_c.sum()
    
    # 如果某组没有样本，返回 0
    if n_t == 0 or n_c == 0:
        return 0.0
    
    # TODO: 计算转化率，并限制在 [0.001, 0.999] 避免 log(0)
    p_t = None  # 你的代码: np.clip(y[mask_t].mean(), 0.001, 0.999)
    p_c = None  # 你的代码: np.clip(y[mask_c].mean(), 0.001, 0.999)
    
    # TODO: 计算 KL 散度
    # kl = p_t * np.log(p_t / p_c) + (1 - p_t) * np.log((1 - p_t) / (1 - p_c))
    kl = None  # 你的代码
    
    return kl


def calculate_euclidean_distance_gain(
    y: np.ndarray,
    t: np.ndarray
) -> float:
    """
    计算欧氏距离增益
    
    ED = (p_t - p_c)^2
    
    TODO: 实现欧氏距离计算
    
    Returns:
        Euclidean distance value
    """
    # 你的代码: 计算 uplift 的平方
    pass

# 测试
kl = calculate_kl_divergence_gain(y_test, t_test)
ed = calculate_euclidean_distance_gain(y_test, t_test)

if kl is not None:
    print(f"KL 散度: {kl:.6f}")
else:
    print("[未完成] 请完成 calculate_kl_divergence_gain 函数")

if ed is not None:
    print(f"欧氏距离: {ed:.6f}")
else:
    print("[未完成] 请完成 calculate_euclidean_distance_gain 函数")

---

## Part 3: 找最佳分裂点

### 分裂的目标

Uplift Tree 的分裂目标是：**最大化分裂后的总增益**

$$Gain_{split} = \frac{n_{left}}{n} \cdot Gain_{left} + \frac{n_{right}}{n} \cdot Gain_{right}$$

### 分裂过程

```
原始节点 (1000 样本, Uplift = 5%)
            |
            ↓ 按 X1 < 0.5 分裂
    ┌───────┴───────┐
    ↓               ↓
左子节点           右子节点
(400 样本)        (600 样本)
Uplift = 15%      Uplift = -2%
                  
成功分出高 Uplift 人群!
```

In [ ]:
# 练习 2.2: 找最佳分裂点

def find_best_split_threshold(
    X: np.ndarray,
    y: np.ndarray,
    t: np.ndarray,
    criterion: str = 'KL'
) -> Tuple[Optional[float], float]:
    """
    找到单变量数据的最佳分裂点
    
    TODO:
    1. 遍历所有可能的分裂阈值
    2. 对每个阈值，计算分裂后的增益
    3. 返回最佳阈值和增益
    
    Args:
        X: 单个特征的值 (1维数组)
        y: 结果
        t: 处理状态
        criterion: 分裂准则 ('KL', 'ED', 'simple')
    
    Returns:
        (best_threshold, best_gain)
    """
    # 获取所有唯一值作为候选分裂点
    unique_values = np.unique(X)
    
    if len(unique_values) < 2:
        return None, 0.0
    
    # TODO: 候选阈值 = 相邻值的中点
    thresholds = None  # 你的代码: (unique_values[:-1] + unique_values[1:]) / 2
    
    best_threshold = None
    best_gain = -np.inf
    
    # 选择增益计算函数
    if criterion == 'KL':
        gain_func = calculate_kl_divergence_gain
    elif criterion == 'ED':
        gain_func = calculate_euclidean_distance_gain
    else:
        gain_func = lambda y, t: calculate_simple_uplift(y, t) ** 2
    
    # TODO: 遍历所有候选阈值
    for threshold in (thresholds if thresholds is not None else []):
        # TODO: 按阈值分裂数据
        left_mask = None   # 你的代码: X <= threshold
        right_mask = None  # 你的代码: X > threshold
        
        # 确保两边都有足够样本 (至少 50 个)
        if left_mask.sum() < 50 or right_mask.sum() < 50:
            continue
        
        # TODO: 计算左右子节点的增益
        left_gain = None   # 你的代码: gain_func(y[left_mask], t[left_mask])
        right_gain = None  # 你的代码: gain_func(y[right_mask], t[right_mask])
        
        # TODO: 加权平均增益
        n_left = left_mask.sum()
        n_right = right_mask.sum()
        n_total = len(y)
        
        weighted_gain = None  # 你的代码: (n_left/n_total)*left_gain + (n_right/n_total)*right_gain
        
        # 更新最佳分裂
        if weighted_gain is not None and weighted_gain > best_gain:
            best_gain = weighted_gain
            best_threshold = threshold
    
    return best_threshold, best_gain

# 生成测试数据
np.random.seed(42)
n = 500
X_test = np.random.randn(n)
T_test = np.random.binomial(1, 0.5, n)

# 异质性效应: X > 0 时 uplift 高
uplift_true = np.where(X_test > 0, 0.3, 0.05)
prob = 0.2 + uplift_true * T_test
Y_test = np.random.binomial(1, prob)

# 找最佳分裂点
threshold, gain = find_best_split_threshold(X_test, Y_test, T_test, criterion='KL')
if threshold is not None:
    print(f"最佳分裂阈值: {threshold:.4f}")
    print(f"分裂增益: {gain:.6f}")
    print(f"\n真实分裂点应该接近 0 (因为我们设置了 X > 0 时 uplift 高)")
else:
    print("[未完成] 请完成 find_best_split_threshold 函数")

---

## Part 4: 叶节点 Uplift 估计

### 叶节点估计什么？

Uplift Tree 的叶节点估计的是：**这个叶节点内样本的平均处理效应**

$$\hat{\tau}_{leaf} = \bar{Y}_{T=1} - \bar{Y}_{T=0}$$

同时，我们还需要记录一些统计信息来评估估计的可靠性。

In [ ]:
# 练习 2.3: 叶节点 Uplift 估计

def estimate_leaf_uplift(
    y: np.ndarray,
    t: np.ndarray
) -> Tuple[float, dict]:
    """
    估计叶节点的 Uplift 及统计信息
    
    TODO: 计算以下信息:
    1. uplift: 处理效应估计
    2. n_treated: 处理组样本数
    3. n_control: 控制组样本数
    4. treated_rate: 处理组转化率
    5. control_rate: 控制组转化率
    
    Returns:
        (uplift, stats_dict)
    """
    mask_t = t == 1
    mask_c = t == 0
    
    n_treated = mask_t.sum()
    n_control = mask_c.sum()
    
    if n_treated == 0 or n_control == 0:
        return 0.0, {
            'n_treated': n_treated,
            'n_control': n_control,
            'treated_rate': 0.0,
            'control_rate': 0.0
        }
    
    # TODO: 计算转化率和 uplift
    treated_rate = None  # 你的代码: y[mask_t].mean()
    control_rate = None  # 你的代码: y[mask_c].mean()
    uplift = None        # 你的代码: treated_rate - control_rate
    
    stats = {
        'n_treated': n_treated,
        'n_control': n_control,
        'treated_rate': treated_rate,
        'control_rate': control_rate
    }
    
    return uplift, stats

# 测试
uplift, stats = estimate_leaf_uplift(Y_test, T_test)
if uplift is not None and stats['treated_rate'] is not None:
    print(f"整体 Uplift: {uplift:.4f}")
    print(f"处理组: {stats['n_treated']} 样本, 转化率 {stats['treated_rate']:.4f}")
    print(f"控制组: {stats['n_control']} 样本, 转化率 {stats['control_rate']:.4f}")
else:
    print("[未完成] 请完成 estimate_leaf_uplift 函数")

---

## Part 5: 构建简单的 Uplift Tree

现在让我们把所有组件整合起来，构建一个简单的 (单层) Uplift Tree！

In [ ]:
# 练习 2.4: 构建简单的 Uplift Tree

class SimpleUpliftTree:
    """
    简单的单层 Uplift Tree
    
    只实现一次分裂，便于理解核心概念
    """
    
    def __init__(self, criterion: str = 'KL', min_samples_leaf: int = 100):
        self.criterion = criterion
        self.min_samples_leaf = min_samples_leaf
        
        # 树结构
        self.split_feature = None
        self.split_threshold = None
        self.left_uplift = None
        self.right_uplift = None
        self.left_stats = None
        self.right_stats = None
    
    def fit(self, X: np.ndarray, T: np.ndarray, Y: np.ndarray):
        """
        训练 Uplift Tree (单次分裂)
        
        TODO:
        1. 遍历所有特征，找最佳分裂
        2. 记录最佳分裂特征和阈值
        3. 计算左右叶节点的 uplift
        """
        n_samples, n_features = X.shape
        
        best_gain = -np.inf
        best_feature = None
        best_threshold = None
        
        # TODO: 遍历所有特征
        for feature_idx in range(n_features):
            # 你的代码: 使用 find_best_split_threshold 函数
            # threshold, gain = find_best_split_threshold(
            #     X[:, feature_idx], Y, T, criterion=self.criterion
            # )
            # if gain > best_gain:
            #     best_gain = gain
            #     best_feature = feature_idx
            #     best_threshold = threshold
            pass
        
        # TODO: 使用最佳分裂点分裂数据，计算叶节点 uplift
        if best_threshold is not None:
            self.split_feature = best_feature
            self.split_threshold = best_threshold
            
            # 分裂数据
            left_mask = X[:, best_feature] <= best_threshold
            right_mask = X[:, best_feature] > best_threshold
            
            # 计算叶节点 uplift
            self.left_uplift, self.left_stats = estimate_leaf_uplift(
                Y[left_mask], T[left_mask]
            )
            self.right_uplift, self.right_stats = estimate_leaf_uplift(
                Y[right_mask], T[right_mask]
            )
        
        return self
    
    def predict(self, X: np.ndarray) -> np.ndarray:
        """
        预测 Uplift
        
        TODO: 根据分裂规则返回对应叶节点的 uplift
        """
        if self.split_threshold is None:
            # 未训练或无有效分裂
            return np.zeros(X.shape[0])
        
        # TODO: 根据分裂特征和阈值，返回对应的 uplift
        predictions = np.zeros(X.shape[0])
        # 你的代码:
        # left_mask = X[:, self.split_feature] <= self.split_threshold
        # predictions[left_mask] = self.left_uplift
        # predictions[~left_mask] = self.right_uplift
        
        return predictions
    
    def get_tree_info(self) -> str:
        """
        返回树的结构信息
        """
        if self.split_threshold is None:
            return "Tree not fitted or no valid split found"
        
        info = f"""
Uplift Tree Structure:
----------------------
Split Rule: X{self.split_feature} <= {self.split_threshold:.4f}

Left Node (X{self.split_feature} <= {self.split_threshold:.4f}):
  Uplift: {self.left_uplift:.4f}
  Treated: {self.left_stats['n_treated']} (rate: {self.left_stats['treated_rate']:.4f})
  Control: {self.left_stats['n_control']} (rate: {self.left_stats['control_rate']:.4f})

Right Node (X{self.split_feature} > {self.split_threshold:.4f}):
  Uplift: {self.right_uplift:.4f}
  Treated: {self.right_stats['n_treated']} (rate: {self.right_stats['treated_rate']:.4f})
  Control: {self.right_stats['n_control']} (rate: {self.right_stats['control_rate']:.4f})
        """
        return info

In [ ]:
# 练习 2.5: 测试 Uplift Tree

def test_uplift_tree():
    """
    测试 Uplift Tree
    
    生成带有异质性效应的数据，验证树能否正确分裂
    """
    np.random.seed(42)
    n = 2000
    
    # 生成特征
    X1 = np.random.randn(n)
    X2 = np.random.randn(n)
    X = np.column_stack([X1, X2])
    
    # 随机分配处理
    T = np.random.binomial(1, 0.5, n)
    
    # 异质性效应: X1 > 0 时 uplift 高，X1 <= 0 时 uplift 低
    true_uplift = np.where(X1 > 0, 0.3, 0.05)
    
    # 基线转化率
    baseline_prob = 0.2
    
    # 生成结果
    prob = baseline_prob + true_uplift * T
    Y = np.random.binomial(1, prob)
    
    # 训练 Uplift Tree
    tree = SimpleUpliftTree(criterion='KL')
    tree.fit(X, T, Y)
    
    # 预测
    pred_uplift = tree.predict(X)
    
    # 打印结果
    if tree.split_threshold is not None:
        print(tree.get_tree_info())
        print(f"\n真实 Uplift (X1>0): {true_uplift[X1 > 0].mean():.4f}")
        print(f"真实 Uplift (X1<=0): {true_uplift[X1 <= 0].mean():.4f}")
    else:
        print("[未完成] 请完成 Uplift Tree 实现")
    
    return tree, pred_uplift, true_uplift, X

# 运行测试
tree, pred_uplift, true_uplift, X = test_uplift_tree()

---

## Part 6: 可视化 Uplift Tree 的效果

In [ ]:
# 可视化

if tree.split_threshold is not None:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # 图1: 真实 Uplift 分布
    scatter1 = axes[0].scatter(X[:, 0], X[:, 1], c=true_uplift, cmap='RdYlGn', s=10, alpha=0.7)
    axes[0].axvline(x=0, color='red', linestyle='--', linewidth=2, label='True boundary (X1=0)')
    if tree.split_feature == 0:
        axes[0].axvline(x=tree.split_threshold, color='blue', linestyle='-', linewidth=2, label=f'Learned boundary (X1={tree.split_threshold:.2f})')
    axes[0].set_xlabel('X1')
    axes[0].set_ylabel('X2')
    axes[0].set_title('True Uplift')
    axes[0].legend()
    plt.colorbar(scatter1, ax=axes[0], label='Uplift')
    
    # 图2: 预测 Uplift
    scatter2 = axes[1].scatter(X[:, 0], X[:, 1], c=pred_uplift, cmap='RdYlGn', s=10, alpha=0.7)
    if tree.split_feature == 0:
        axes[1].axvline(x=tree.split_threshold, color='blue', linestyle='-', linewidth=2, label=f'Split at X1={tree.split_threshold:.2f}')
    axes[1].set_xlabel('X1')
    axes[1].set_ylabel('X2')
    axes[1].set_title('Predicted Uplift (by Uplift Tree)')
    axes[1].legend()
    plt.colorbar(scatter2, ax=axes[1], label='Uplift')
    
    plt.tight_layout()
    plt.show()
else:
    print("请先完成上面的练习")

---

## 思考题

### 1. 为什么 Uplift Tree 需要特殊的分裂准则，而不能使用传统的信息增益或基尼系数?

**你的答案:**


### 2. KL 散度、欧氏距离、卡方统计量这三种分裂准则有什么区别? 各自适用于什么场景?

**你的答案:**

KL 散度:
-

欧氏距离:
-

卡方统计量:
-

### 3. Uplift Tree 的叶节点估计的是什么? 如何解释叶节点的值?

**你的答案:**


### 4. 如果某个叶节点只有处理组样本或只有控制组样本，应该如何处理?

**你的答案:**


### 5. Uplift Tree 相比 Meta-Learners 有什么优势和劣势?

**你的答案:**

优势:
-

劣势:
-

### 6. 在什么情况下应该使用 Uplift Tree，什么情况下应该使用 Meta-Learners?

**你的答案:**


---

## 总结

| 概念 | 要点 |
|------|------|
| **Uplift Tree** | 专门为因果推断设计的决策树 |
| **分裂目标** | 最大化子节点间处理效应的差异 |
| **KL 散度** | 衡量处理组和控制组分布差异 |
| **叶节点估计** | 处理组转化率 - 控制组转化率 |
| **优势** | 可解释性强，能处理高维交互 |

### 下一步

在下一个练习中，我们将学习如何 **评估 Uplift 模型** —— Qini 曲线和 AUUC！

---

*恭喜你完成了 Uplift Tree 的学习！* 🎉